In [10]:
## WORKS ON PYTHON3.6 AND ABOVE

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import bs4
import re
import json
import math
import numpy as np
import h5py
import deepdish as dd

In [2]:
CITY = "jaipur"
CITY_ID = "10" # JAIPUR 10 | NCR 1 | KOLKATA 2 | MUMBAI 3 | BANGALORE 4 | PUNE 5 | HYDERABAD 6 | CHENNAI 7
API_USER_KEY = "GET YOUR OWN API USER KEY"
# API KEY FROM ZOMATO WITH 1000 CALLS/DAY

In [3]:
url = f"https://www.zomato.com/{CITY}/restaurants?page=1"
url_i = f"https://www.zomato.com/{CITY}/restaurants?page="

In [4]:
user_agent = "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"

In [5]:
headers = {'user-agent': user_agent,'user-key':API_USER_KEY}

In [6]:
REQ_SEARCH_URL = lambda x=0: f'https://developers.zomato.com/api/v2.1/search?entity_id={CITY_ID}&entity_type=city&start={x}'
REQ_RESTAURANT_URL = lambda x=0: f'https://developers.zomato.com/api/v2.1/restaurant?res_id={x}'

In [7]:
TOTAL_NUMBER_OF_RESTAURANTS = int(json.loads(requests.get(REQ_SEARCH_URL(),headers=headers).text)['results_found'])

In [8]:
## GET URLS OF ALL RESTAURANTS THROUGH SCRAPING  (DOES THIS METHOD GET ALL THE RESTAURANTS???)

In [27]:
URLs = []
TOTAL_BLOCKS = math.ceil(TOTAL_NUMBER_OF_RESTAURANTS/15) # IS THE DIVISION ALRIGHT?
for i in range(1,TOTAL_BLOCKS):
#for i in range(1,2):
    print(f"Block: {i}",end="\r")
    url_ = url_i + str(i)
    r = requests.get(url_,headers=headers)
    doc = bs(r.text,'html.parser')
    search_results = doc.find_all(attrs={"id":"orig-search-list"})
    divs = []
    for div in search_results:
        divs.append(div)
    for div in divs[0]:
        if isinstance(div,bs4.element.Tag):
            url = div.find_all("a",{"class":"result-title hover_feedback zred bold ln24 fontsize0"})[0]['href']
            ## IF FINDING <a> THROUGH THIS CLASS NAME DOESN'T WORK, FIND THE CORRECT CLASS NAME
            URLs.append(url)

In [318]:
## GET ALL RESTAURANT IDs FROM THE URL

In [10]:
RES_IDs = []
for i in range(len(URLs)):
    print(f"URLs INDEX: {i}",end='\r')
    r = requests.get(URLs[i],headers=headers)
    d = bs(r.text,'html.parser')
    e = d.find_all("script")
    f = e[14]
    g = re.sub(' ','',f.text)
    i = g.split("\n")[1].split("=")[1].split("\"")[1]
    RES_IDs.append(i)

In [17]:
## GET RESTAURANT'S DETAILS THROUGH API

In [13]:
REST = dict()
errors_on = []
def get_restaurants(res_ids):
    print("TOTAL:",len(res_ids))
    for res_id_idx in range(len(res_ids)):
        print("                                                                         ",end='\r')
        print("RES_ID_COUNT: " + str(res_id_idx+1) + " | " + "RES_ID: " + res_ids[res_id_idx],end='\r')
        r = requests.get(REQ_RESTAURANT_URL(res_ids[res_id_idx]),headers=headers)
        j = json.loads(r.text)
        try:
            res_ = {j['id']: j}
            REST.update(res_)
        except:
            errors_on.append(res_id_idx)
    print("                                               ",end='\r')
    print("RES_ID_COUNT: " + str(res_id_idx+1) + " | " + "DONE",end='\r')

In [14]:
get_restaurants(RES_IDs[:500])

TOTAL: 500


In [23]:
## SAVE ALL THE STUFF
np.save(f"URLS_{CITY}",URLs)
np.save(f"IDS_{CITY}",RES_IDs)
#np.save("RESTAURANTS",REST)
dd.io.save(f"RESTAURANTS_{CITY}.h5",REST)

In [19]:
## LOAD ALL THE STUFF
RES_IDs = np.load(f"IDS_{CITY}.npy")
URLs = np.load(f"URLS_{CITY}.npy")
#REST = np.load("RESTAURANTS.npy")
REST = dd.io.load(f"RESTAURANTS_{CITY}.h5")